# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data_quantity = data.groupby(['CustomerID', 'ProductName']).agg({"Quantity":"sum"})
data_quantity.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
data_pivot = pd.pivot_table(data_quantity, values='Quantity', index=['ProductName'],columns=['CustomerID']).fillna(0)
data_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
similarities = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'euclidean'))), index=data_pivot.columns, columns=data_pivot.columns)

similarities.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
list_customers = similarities[33].sort_values(ascending=False)[1:6].index.tolist()
print(list_customers)

[264, 3535, 3317, 2503, 3305]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
data_quantity2 = data_quantity.reset_index()
data_quantity2 = data_quantity2.loc[data_quantity2['CustomerID'].isin(list_customers)]
data_quantity2.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [37]:
ranked_products = data_quantity2.groupby(['ProductName']).sum()['Quantity'].reset_index()
ranked_products = ranked_products.sort_values(['Quantity'],ascending=False)
ranked_products.head()

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [63]:
data_merge = ranked_products.merge(data_pivot, on="ProductName")
data_merge = data_merge[data_merge[33]==0][["ProductName", "Quantity"]]
data_merge.head()[0:5]

,ProductName,Quantity
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
3,Soup - Campbells Bean Medley,3
4,Wine - Blue Nun Qualitatswein,3
6,Chicken - Soup Base,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [88]:
dicc={}
list_customers_user = list(set(data['CustomerID']))

def Recommendation(user):
    list_customers = similarities[user].sort_values(ascending=False)[1:6].index.tolist()
    data_quantity2 = data_quantity.reset_index()
    data_quantity2 = data_quantity2.loc[data_quantity2['CustomerID'].isin(list_customers)]
    ranked_products = data_quantity2.groupby(['ProductName']).sum()['Quantity'].reset_index()
    ranked_products = ranked_products.sort_values(['Quantity'],ascending=False)
    data_merge = ranked_products.merge(data_pivot, on="ProductName")
    data_merge_2= data_merge[data_merge[user]==0]["ProductName"]
    return list(data_merge_2[:5])

In [89]:
dicc = {i:Recommendation(i) for i in list_customers_user}

In [100]:
dicc[200]

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [119]:
data_recommendation = pd.DataFrame(dicc).transpose().reset_index()
data_recommendation.columns = ["CustomerID","Product 1", "Product 2", "Product 3", "Product 4", "Product 5"]
data_recommendation.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
1,59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
2,92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
3,49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
4,18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [120]:
similarities = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'cityblock'))), index=data_pivot.columns, columns=data_pivot.columns)

similarities.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.008000,0.009901,0.009174,0.008403,0.008929,0.007752,0.008850,0.008621,0.008696,...,0.000564,0.000596,0.000556,0.000674,0.000542,0.000569,0.000556,0.000554,0.000476,0.000534
200,0.008000,1.000000,0.008130,0.007874,0.006993,0.007576,0.007752,0.007752,0.008065,0.008130,...,0.000563,0.000593,0.000557,0.000675,0.000538,0.000567,0.000558,0.000553,0.000473,0.000531
264,0.009901,0.008130,1.000000,0.008547,0.008850,0.008475,0.007874,0.008403,0.008621,0.008264,...,0.000566,0.000596,0.000556,0.000678,0.000543,0.000576,0.000559,0.000557,0.000479,0.000533
356,0.009174,0.007874,0.008547,1.000000,0.008000,0.008197,0.007752,0.009174,0.008065,0.008130,...,0.000564,0.000595,0.000556,0.000676,0.000540,0.000571,0.000553,0.000556,0.000478,0.000534
412,0.008403,0.006993,0.008850,0.008000,1.000000,0.008065,0.007752,0.008850,0.007937,0.008264,...,0.000561,0.000599,0.000553,0.000674,0.000540,0.000570,0.000552,0.000555,0.000477,0.000536


In [121]:
dicc2={}
list_customers_user = list(set(data['CustomerID']))

def Recommendation(user):
    list_customers = similarities[user].sort_values(ascending=False)[1:6].index.tolist()
    data_quantity2 = data_quantity.reset_index()
    data_quantity2 = data_quantity2.loc[data_quantity2['CustomerID'].isin(list_customers)]
    ranked_products = data_quantity2.groupby(['ProductName']).sum()['Quantity'].reset_index()
    ranked_products = ranked_products.sort_values(['Quantity'],ascending=False)
    data_merge = ranked_products.merge(data_pivot, on="ProductName")
    data_merge_2= data_merge[data_merge[user]==0]["ProductName"]
    return list(data_merge_2[:5])

In [123]:
dicc2 = {i:Recommendation(i) for i in list_customers_user}

In [124]:
dicc2[200]

['Soup - Campbells Bean Medley',
 'Pomello',
 'Bay Leaf',
 'Pork - Kidney',
 'Muffin - Carrot Individual Wrap']

In [125]:
data_recommendation_2 = pd.DataFrame(dicc).transpose().reset_index()
data_recommendation_2.columns = ["CustomerID","Product 1", "Product 2", "Product 3", "Product 4", "Product 5"]
data_recommendation_2.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,83973,"Pork - Bacon, Double Smoked","Nut - Chestnuts, Whole",Olive - Spread Tapenade,Butter - Unsalted,Beef Ground Medium
1,59399,"Pepper - Black, Whole",Soupfoamcont12oz 112con,"Pork - Bacon, Double Smoked",Soup - Campbells Bean Medley,"Salsify, Organic"
2,92168,Wine - Blue Nun Qualitatswein,Muffin Batt - Blueberry Passion,Knife Plastic - White,Chinese Foods - Chicken,"Wine - Magnotta, Merlot Sr Vqa"
3,49159,Wine - Blue Nun Qualitatswein,Butter - Unsalted,Tia Maria,Veal - Sweetbread,Wine - Chardonnay South
4,18441,Anchovy Paste - 56 G Tube,Flour - Pastry,Foam Cup 6 Oz,"Garlic - Primerba, Paste",Butter - Unsalted
